### Author: Michele Grossi - IBM Italy

In [ ]:
# %matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

## Leveraging Provider information

In [ ]:
#Create the Provider object using the IBMQ interface 
#provider = IBMQ.get_provider(hub='ibm-q-research', group='Michele-Grossi', project='main')
provider = IBMQ.get_provider()
#Query the list of backends available to your account
provider.backends()

In [ ]:
#Import the least_busy function
from qiskit.providers.ibmq import least_busy

#Identify the least busy devices 
#smaller than 6 qubits and not a simulator
small_devices = provider.backends(filters=lambda x: x.configuration().n_qubits < 6 and not x.configuration().simulator)

#Identify the least busy devices 
#larger than 6 qubits and not a simulator
large_devices = provider.backends(filters=lambda x: x.configuration().n_qubits > 6 and not x.configuration().simulator)

#Print the least busy devices
print('The least busy small devices: {}'.format(least_busy(small_devices)))
print('The least busy large devices: {}'.format(least_busy(large_devices)))

In [ ]:
#Set ibmq_valencia as the backend, or whichever backend you wish
backend = provider.get_backend('ibmq_valencia')
#or 
backend = provider.backends.ibmq_valencia
#Confirm this is the backend selected by querying for its name,
backend.name()

In [ ]:
#View the status of the backend
status = backend.status()
is_operational = status.operational
jobs_in_queue = status.pending_jobs
print('is_operational: {0}, jobs_in_queue: {1}'.format(is_operational,jobs_in_queue))

In [ ]:
#View backend properties
backend

In [ ]:
#Run a few jobs on this backend to generate jobs on the backend
qc = QuantumCircuit(1,1)
qc.h(0)
qc.measure_all()
for i in range(0,3):
    result = execute(qc, backend, shots=1024).result()

In [ ]:
#List out the last 3 jobs we ran on the device
for executed_job in backend.jobs(limit=3):
    print('Job id: '
          + str(executed_job.job_id()) + ', '  
          + str(executed_job.status()))

In [ ]:
#From the previous output of executed jobs, enter its job id.
job = backend.retrieve_job(executed_job.job_id())
job.status()

In [ ]:
job.backend()

### Monitoring and tracking jobs

In [ ]:
#Import the Qiskit Jupyter tools 
from qiskit.tools import jupyter
#Initialize the job tracker to automatically track all jobs
%qiskit_job_watcher

In [ ]:
backend = provider.get_backend('ibmq_athens')
#Create a simple circuit
qc = QuantumCircuit(1)
qc.h(0)
qc.measure_all()
#Execute the circuit on the backend
job = execute(qc, backend)

In [ ]:
#Display the list of all available backends and provide 
#a brief overview of each 
%qiskit_backend_overview

## Optimizing circuits using the Transpiler

In [ ]:
#Basic Toffoli gate,
qc = QuantumCircuit(3)
qc.ccx(0,1,2)
qc.draw()

In [ ]:
qc_decomposed = qc.decompose()
qc_decomposed.draw()

In [ ]:
#Basic circuit with a single and multi-qubit gates
qc = QuantumCircuit(4)
qc.h(0)
qc.cx(0,1)
qc.cx(0,2)
qc.cx(0,3)
qc.draw()

In [ ]:
#Print the depth of both inital and decomposed circuit
print('Initial circuit depth: ', qc.depth())
print('Decomposed circuit depth: ', qc_decomposed.depth())
#Get the number of operators in initial circuit
print('Initial circuit operation count: ', qc.count_ops())
#Get the number of operators in decomposed circuit
print('Decomposed circuit operation count: ', qc_decomposed.count_ops())

In [ ]:
# Get the backend device: ibmq_rome 
backend_athens = provider.get_backend('ibmq_athens')
# Launch backend viewer of ibmq_rome
backend_athens

In [ ]:
# Get the backend device: ibmq_16_melbourne
backend_melbourne = provider.get_backend('ibmq_16_melbourne')
# Launch backend viewer of ibmq_16_melbourne
backend_melbourne

In [ ]:
# Visualize the coupling directional map between the qubits 
plot_gate_map(backend_athens, plot_directed=True)

In [ ]:
# Quantum circuit with a single and multi-qubit gates
qc = QuantumCircuit(4)
qc.h(0)
qc.cx(0,1)
qc.cx(0,2)
qc.cx(0,3)
qc.draw()

In [ ]:
# Transpile the circuit with an optimization level = 0
qc_athens_0 = transpile(qc, backend_athens, seed_transpiler=10258, optimization_level=0)
# Print out the depth of the circuit
print('Depth:', qc_athens_0.depth())
# Plot the resulting layout of the quantum circuit after Layout
plot_circuit_layout(qc_athens_0, backend_athens)

In [ ]:
qc_athens_0.draw()

In [ ]:
# View the transpiled circuit with an optimization level = 0
qc_melbourne_0 = transpile(qc, backend_melbourne, seed_transpiler=10258, optimization_level=0)
print('Depth:', qc_melbourne_0.depth())
plot_circuit_layout(qc_melbourne_0, backend_melbourne)

In [ ]:
qc_melbourne_0.draw()

In [ ]:
# Transpile the circuit with the optimization level = 3
qc_transpiled_athens = transpile(qc, backend_athens, optimization_level=3)
# Print the depth of the transpiled circuit
print('Depth:', qc_transpiled_athens.depth())
# Print the number of operations of the transpiled circuit
print('Ops count: ', qc_transpiled_athens.count_ops())
# Plot the layout mapping of the transpiled circuit

In [ ]:
qc_transpiled_athens.draw()

In [ ]:
# Transpile the quantum circuit with the optimization level = 3
qc_transpiled_melbourne = transpile(qc, backend_melbourne, optimization_level=3)
# Get the depth and operation count of the transpiled circuit. 
print('Depth:', qc_transpiled_melbourne.depth())
print('Ops count: ', qc_transpiled_melbourne.count_ops())
# Print the circuit layout
plot_circuit_layout(qc_transpiled_melbourne, backend_melbourne)

In [ ]:
qc_transpiled_melbourne.draw()

In [ ]:
# View the backend coupling map, displayed as CNOTs (Control-Target)
backend = provider.get_backend('ibmqx2')
# Extract the coupling map from the backend
ibmqx2_coupling_map = backend.configuration().coupling_map
# List out the extracted coupling map
ibmqx2_coupling_map

In [ ]:
# Transpile the custom circuit using only the coupling map. 
# Set the backend to None so it will force using the coupling map provided.
qc_custom = transpile(qc, backend=None, coupling_map=ibmqx2_coupling_map)
# Draw the resulting custom topology circuit.
qc_custom.draw(output='mpl')

In [ ]:
# Create our own coupling map (custom topology)
custom_linear_topology = [[0,1],[1,2],[2,3],[3,4]]
# Set the coupling map to our custom linear topology
qc_custom = transpile(qc, backend=None, coupling_map=custom_linear_topology)
# Draw the resulting circuit.
qc_custom.draw(output='mpl')

The result from the preceding circuit code is clearly not ideal. The circuit required many gates and is quite deep, which increases the risk of having noisy results. This is a good illustration of the importance of optimizers, which handle many of these potential issues. It's no surprise why there is a lot of research in identifying better ways to optimize circuits to avoid inefficient and noisy circuits

## Understanding passes and pass managers

Now that we have a better understanding of passes and how some of them help the transpiler generate optimal circuits, we just need to conclude with the pass manager. The pass manager is what allows the passes to communicate with each other, and also schedules which passes should execute first.


Passes are generally used to transform circuits so that they are set up to perform as optimally as desired. There are five general types of passes that transform circuits:

-- Layout Selection determines how the qubit layout mapping will align with the selected backend configuration.
-- Routing maps the placement of swap gates onto the circuit based on the selected swap mapping type
-- Basis Change offers various ways to decompose or unroll the gates down to the basis gates of the backend or using the circuit's decomposition rules.
-- Optimizations optimizes the gates themselves by either removing redundant gates
-- Circuit Analysis provides circuit information, such as the depth, width, number of operations, and other details about the circuit,
-- Additional passes are those that offer some other form of optimization, such as the various check maps, which check whether the layout of the CNOT gates are in the direction stated in the coupling maps and rearrange the directions if needed.

[L.Loredo, M.Grossi, Learn Quantum Computing with Python and IBM Q Experience: A Hands-On Introduction to Quantum Computing and Writing Your Own Quantum Programs with Python, https://books.google.it/books?id=jKy3zQEACAAJ ]


In [ ]:
# Import the transpiler passes object
from qiskit.transpiler import passes
# List out all the passes available
print(dir(passes))

In [ ]:
# Import the PassManager and a few Passes
from qiskit.transpiler import PassManager, CouplingMap
from qiskit.transpiler.passes import TrivialLayout, BasicSwap
# Create a BasicSwap based on the ibmqx2 coupling map we used earlier
basic_swap = BasicSwap(CouplingMap(ibmqx2_coupling_map))
#Add the BasicSwap to the PassManager
pm = PassManager(basic_swap)
# Run the PassManager and draw the results
new_qc = pm.run(qc)
new_qc.draw(output='mpl')

In [ ]:
# Create a TrivialLayout based on the ibmqx2 coupling map
trivial = TrivialLayout(CouplingMap(ibmqx2_coupling_map))
# Append the TrivialLayout to the PassManager
pm.append(trivial)
# Run the PassManager and draw the resulting circuit
tv_qc = pm.run(qc)
tv_qc.draw(output='mpl')

## Visualizing and enhancing circuit graphs

In [ ]:
# Define the style to render the circuit and components
style = {'backgroundcolor': 'lightblue','gatefacecolor': 'white', 'gatetextcolor': 'black', 'fontsize': 14}# Draw the mpl with the specified style
qc.draw(output='mpl', style=style)

### Directed Acyclic Graph of a circuit
If you break down a circuit into composites, you can then render each composite as a DAG

In [ ]:
# Import the Circuit to DAG converter
from qiskit.converters import circuit_to_dag
# Import the DAG drawer
from qiskit.tools.visualization import dag_drawer
# Convert the circuit into a DAG
dag = circuit_to_dag(qc)
# Draw the DAG of the circuit
dag_drawer(dag)

The DAG can help illustrate the flow and expected paths of the circuit. For example, the preceding graph starts at the top with the qubits in green, then following the graph, we see that each qubit is operated upon by the specified operation represented by the nodes and the applied qubits by the edge label between nodes. The graph terminates at the end in red, where the measurement applied on the qubit is mapped to the specified classical bit, represented by the parameter values.

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright